## Text Summarization

В данном задании мы будем строить саммари из текста, используя алгоритм LexRank (https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume22/erkan04a-html/erkan04a.html)

In [88]:
import pandas as pd
import numpy as np
import os
import math
from collections import Counter
import nltk.data
from collections import defaultdict

Загрузим документы из файлов.

In [2]:
data_dir = './data'

In [3]:
documents = []
for file in os.listdir(data_dir):
    file_name = data_dir + '/' + file
    with open(file_name) as f:
        data = f.read()
        documents.append(data.strip())

Посмотрим, как выглядят документы:

In [4]:
len(documents)

10

In [5]:
documents[0]

"Bob Livingston, the incoming speaker of the House, took no public role Friday as the debate unfolded on whether to impeach President Clinton. His previous 24 hours had been his most visible in the month since his party nominated him as speaker and perhaps the most excruciating in his public career: He spent most of the day in a chaotic procedural wrangle over the terms of Friday's debate; he ended it by telling his Republican colleagues that he had had extramarital affairs during his 20-year tenure in Congress. His disclosure followed an investigation by Larry Flynt, publisher of Hustler, the sex magazine, who said Friday at a news conference in Beverly Hills, Calif., that his publication had learned that Livingston had had adulterous affairs during the last 10 years. Flynt said earlier this year that he wanted to expose the ``hypocrisy'' of those in Washington who are investigating Clinton and in October offered $1 million to anyone who could prove they had had affairs with members o

In [6]:
def get_sentences(doc):
    """
    Get sentences from document

    Inputs:
    - text: string

    Returns:
    - list of sentences
    """
    list_sentences = doc.split('.')
        
    
    
    return list_sentences

In [7]:
#decorators
def to_utf8(text):
    if isinstance(text, unicode): text = text.encode('utf8')
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, unicode): text = text.decode('utf8')
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

In [8]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = unicode()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = unicode()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

Далее нужно разделить каждое предложение на токены (слова), при этом можно удалить пунктуацию, стоп-слова, выполнить стемминг.

In [41]:
def sentence_words(sentence):
    """
    Get words from sentence
    
    Perform normalization, remove punctuation, remove stop_words, etc...

    Inputs:
    - sentence: string - sentence text

    Returns:
    - list of strings
    """
    sentence_terms = None
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    
    tokens = []
    sentence_terms = pymorphy_tokenizer(sentence)
    for word in sentence_terms:
        tokens.append(word)
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    return tokens

In [42]:
words  = sentence_words(text[0])
for word in words:
    print ('{}'.format(word))   

bob
livingston
the
incoming
speaker
of
the
house
took
no
public
role
friday
as
the
debate
unfolded
on
whether
to
impeach
president
clinton


In [117]:
sentences = list(())
for txt in text:
    words = sentence_words(txt)
    sentences.append(words)

Нам необходима функция для подсчета tf слов в каждом предложении:

In [118]:
def compute_tf(sentences):
    """
    Compute TF for each term in sentece for all sentences

    Inputs:
    - sentences: list of tokenized sentences

    Returns:
    - list of dicts of tf values
    """
    tf_values = map(Counter, sentences)

    tf_metrics = []
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    for i, sentence in enumerate(sentences): 
        data = dict() 
        for word in sentence: 
            data.update({word : tf_values[i][word]/float(len(sentence))}) 
            tf_metrics.append(data) 
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return tf_metrics

Так же нам необходима функция для подсчета IDF всех токенов из текста:

In [119]:
def compute_idf(sentences):
    idf_metrics = {}
    sentences_count = len(sentences)
    
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    all_tokens_set = set([item for sublist in sentences for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, sentences)
        idf_metrics[tkn] = 1 + np.log(sentences_count/(sum(contains_token)))
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    return idf_metrics

Для того, чтобы определить похожесть двух предложений, нам нужно использовать какую-то метрику похожести. Мы будем использовать модицифированное косинусное расстояние.

In [ ]:
def singl_compute_tf(sentence):
     for word in sentence: 
        data.update({word : tf_values[i][word]/float(len(sentence))}) 
        tf_metrics.append(data) 

In [120]:
tf_list = compute_tf(sentences)

In [124]:
tf_list[0][u'as']

0.043478260869565216

In [129]:
sentences

[[u'bob',
  u'livingston',
  u'the',
  u'incoming',
  u'speaker',
  u'of',
  u'the',
  u'house',
  u'took',
  u'no',
  u'public',
  u'role',
  u'friday',
  u'as',
  u'the',
  u'debate',
  u'unfolded',
  u'on',
  u'whether',
  u'to',
  u'impeach',
  u'president',
  u'clinton'],
 [u'his',
  u'previous',
  u'24',
  u'hours',
  u'had',
  u'been',
  u'his',
  u'most',
  u'visible',
  u'in',
  u'the',
  u'month',
  u'since',
  u'his',
  u'party',
  u'nominated',
  u'him',
  u'as',
  u'speaker',
  u'and',
  u'perhaps',
  u'the',
  u'most',
  u'excruciating',
  u'in',
  u'his',
  u'public',
  u'career',
  u'he',
  u'spent',
  u'most',
  u'of',
  u'the',
  u'day',
  u'in',
  u'a',
  u'chaotic',
  u'procedural',
  u'wrangle',
  u'over',
  u'the',
  u'terms',
  u'of',
  u'friday',
  u's',
  u'debate',
  u'he',
  u'ended',
  u'it',
  u'by',
  u'telling',
  u'his',
  u'republican',
  u'colleagues',
  u'that',
  u'he',
  u'had',
  u'had',
  u'extramarital',
  u'affairs',
  u'during',
  u'his',
  u'2

In [266]:
def cosine_similarity(sentence1, sentence2, tf1, tf2, idf_metrics):
    """
    Compute idf-modified-cosine distance between two sentences.
    
    Inputs:
    - sentence1: list of terms of sentence 1
    - sentence2: list of terms of sentence 2
    - tf1: dict of term frequencies of sentence 1
    - tf2: dict of term frequencies of sentence 2
    - idf_metrics: list of inverted document frequencies

    Returns:
    - modified cosine similarity
    """
    similarity = 0.0
    
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    
    a = set(sentence1)
    b = set(sentence2)
    c = a.intersection(b)
    
    numerator = np.sum((tf1[word]*tf2[word]) * np.square(idf_metrics[word]) for word in c)

    denominator=np.sqrt(np.sum(np.square(tf1[word1]*idf_metrics[word1]) for word1 in a))*\
                np.sqrt(np.sum(np.square(tf2[word2]*idf_metrics[word2]) for word2 in b))
        
    
    
    similarity = numerator/float(denominator)
    
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    
    return similarity

In [267]:
cosine_similarity(sentences[0], sentences[1], compute_tf([sentences[0]])[0], compute_tf([sentences[1]])[0], compute_idf(sentences) )

0.13146040393268776

Создадим матрицу размером NxN, где N - это количество предложений. В каждой ячейки i,j - будем записывать похожесть i-го предложения на j-ое.

In [297]:
def create_matrix(sentences, threshold = 0.01, tf_metrics = 'cosine', idf_metrics = 'cosine'):
    """
    Creates matrix of shape |sentences|×|sentences|.
    
    Inputs:
    - sentences: list of sentences
    - threshold: threshold value for selecting edges
    - tf_metrics: list of TF metrics for each sentence
    - idf_metrics: list of inverted document frequencies

    Returns:
    - matrix of similarities
    """
    sentences_count = len(sentences)-1
    matrix = np.zeros((sentences_count, sentences_count))
    degrees = np.zeros((sentences_count, ))
    
    idf_metrics = compute_idf(sentences)
    
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    for i in range(sentences_count):
        for j in range(sentences_count):
            temp_value = cosine_similarity(sentences[i], 
                                         sentences[j],  
                                         compute_tf([sentences[i]])[0], 
                                         compute_tf([sentences[j]])[0], 
                                         idf_metrics)
            if temp_value<threshold:
                matrix[i][j] = 0
            else:
                matrix[i][j] = temp_value
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return matrix

In [298]:
A = create_matrix(sentences)

In [299]:
A

array([[ 1.        ,  0.1314604 ,  0.04514347, ...,  0.10283431,
         0.03120173,  0.        ],
       [ 0.1314604 ,  1.        ,  0.14443974, ...,  0.01516118,
         0.        ,  0.        ],
       [ 0.04514347,  0.14443974,  1.        , ...,  0.04402928,
         0.02039254,  0.0957373 ],
       ..., 
       [ 0.10283431,  0.01516118,  0.04402928, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.03120173,  0.        ,  0.02039254, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.0957373 , ...,  0.        ,
         0.        ,  1.        ]])

Реализуем power-метод:

In [343]:
a = np.array([1,2,3])
b = np.array([1,2,3])
a.dot(b)

14

In [405]:
def power_method(matrix, epsilon):
    """
    Perform Power-method.
    
    Inputs:
    - matrix: matrix of sentences similarities
    - epsilon: stop when values changes less then epsilon
    - tf_metrics: list of TF metrics for each sentence
    - idf_metrics: list of inverted document frequencies

    Returns:
    - vector of sentence's importancies
    """
    transposed_matrix = matrix.T
    sentences_count = len(matrix)
    p_vector = np.array([1.0 / sentences_count] * sentences_count)
    delta = 1.0

    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    
    while (delta>epsilon):
        p_prev = p_vector
        p_vector = transposed_matrix.dot(p_prev)
        delta = (p_vector - p_prev).dot(p_vector - p_prev)
    
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################

    return p_vector

In [406]:
p_vec = power_method(A, 0.1)

In [407]:
p_vec

array([ 0.0956048 ,  0.10402652,  0.07800694,  0.07192362,  0.09295337,
        0.07672031,  0.09774602,  0.06059245,  0.06016124,  0.06479622,
        0.1002883 ,  0.07255102,  0.09374867,  0.08061369,  0.07129237,
        0.0494492 ,  0.03502812,  0.06630681,  0.08296964,  0.09210437,
        0.07038013,  0.03502812,  0.07299352,  0.03697844,  0.0600822 ,
        0.0495949 ,  0.05293147,  0.0540338 ,  0.05133441,  0.02941176,
        0.08899941,  0.06744371,  0.05504874,  0.03917764])

In [384]:
np.argsort(p_vec)[:-1]

array([29, 16, 21, 23, 33, 15, 25, 28, 26, 27, 32, 24,  8,  7,  9, 17, 31,
       20, 14,  3, 11, 22,  5,  2, 13, 18, 30, 19,  4, 12,  0,  6, 10])

По полученым оценкам важности каждого предложения выделим k-самых важных.

In [412]:
def get_top_sentences(sentences, values, k):
    """
    Get K top rated sentences from all sentences.
    
    Inputs:
    - sentences: list of sentences
    - values: list of computed sentence importancies
    - k: number of sentences to extract

    Returns:
    - list of sentences
    """
    top_sentences = []
    
    #############################################################################
    #                                 YOUR CODE                                 #
    #############################################################################
    dictionary = defaultdict()
    for imp,sentence in zip(values,sentences):
        dictionary[sentence] = imp
        
    top_sentences = sorted(dictionary, key=dictionary.get, reverse=True)[:k]
    
    #############################################################################
    #                              END OF YOUR CODE                             #
    #############################################################################
    
    return top_sentences

Построим саммари:

In [413]:
for document in documents:
    sentences = get_sentences(document)
    sentences_tokenized = [sentence_words(x) for x in sentences]
    tf_metrics = compute_tf(sentences_tokenized)
    idf_metrics = compute_idf(sentences_tokenized)
    
    matrix = create_matrix(sentences_tokenized, 0.3, tf_metrics, idf_metrics)
    sentence_values = power_method(matrix, 0.001)
    
    print(get_top_sentences(sentences, sentence_values, 3))

[' His disclosure followed an investigation by Larry Flynt, publisher of Hustler, the sex magazine, who said Friday at a news conference in Beverly Hills, Calif', ", which has done work for the president's private lawyers", ' He spent most of the day in a private office off the chamber, avoiding the gauntlet of reporters who lay in wait for him between the House and his regular office across the street']
["'' Presidential resignation was in order, he continued, and then came the shouts of ``You resign! You resign!'' And as Livingston closed the circle on his career, he stunned the place into a collective breath of disbelief and somehow almost threatened to reduce the dark historic issue before the House _ the impeachment of the president _ into a matter of anticlimax", ' Shockingly, stunningly, Livingston did just that', ' They had to stand, almost staggering to their feet, to join the Republicans in a prolonged, emotional ovation for Livingston and his undeniable self-sacrifice']
[' T

/Users/romandegtyarev/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in subtract


[" Livingston's decision was said to be driven in part by the anger of a group of about a dozen conservatives in his party who were disillusioned that he had withheld news of his affairs from them", "'' As the leaders tried Saturday to hold the party together on and off the House floor, most Republicans stuck to their prepared scripts on the floor in favor of impeaching the president", ", who admitted to his own adulterous affair earlier this year, said after Livingston's resignation: ``Those of us who are sinners must feel wretched today"]
[" ``It breaks your heart because we're all subject to human frailties,'' said Asa Hutchinson, R-Ark", "'' ``During my 33-year marriage to my wife, Bonnie, I have on occasion strayed from my marriage, and doing so nearly cost me my marriage and family,'' Livingston said in his brief prepared statement", " ``We've got a duty to do under the Constitution"]
[' At a time when events in the world and the challenges at home demand that we stand united, ce